In [1]:
# All vid gen techniques are described in
# https://huggingface.co/docs/diffusers/main/en/using-diffusers/text-img2vid
# https://huggingface.co/docs/diffusers/v0.31.0/using-diffusers/text-img2vid

# clone our repo and install requirements in colab
%cd /content/
!rm -rf LSVD
!git clone -b computerVision https://github.com/davidg-h/LSVD.git
%cd LSVD
%pip install -r requirements.txt
!apt install ffmpeg
!git pull origin Inference

/content
Cloning into 'LSVD'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 75 (delta 24), reused 58 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (75/75), 4.64 MiB | 8.02 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/content/LSVD
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Atte

In [1]:
fps = 7
num_frames = 49 #49 / 7fps = ~6s
decode_chunk_size=2
prompt = "A space rocket with trails of smoke behind it launching into space from the desert"
negt = "bad quality, worse quality, low resolution"

------------- Text to Image Gen -------------

In [6]:
# https://huggingface.co/tasks/text-to-image
# https://huggingface.co/black-forest-labs/FLUX.1-dev
# https://huggingface.co/docs/diffusers/using-diffusers/conditional_image_generation
# https://huggingface.co/blog/if
# https://huggingface.co/docs/diffusers/en/api/pipelines/stable_diffusion/text2img
# https://huggingface.co/docs/diffusers/v0.16.0/en/api/pipelines/stable_diffusion/text2img

import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

model_id = "stabilityai/stable-diffusion-2"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.enable_model_cpu_offload()

image = pipe(prompt).images[0]

image.save("reference.png")

# cleanup
pipe.to("cpu")
del pipe
torch.cuda.empty_cache()

scheduler/scheduler_config.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

------------- Text/Img to Video I2VGen -------------

In [2]:
import torch
from diffusers import I2VGenXLPipeline
from diffusers.utils import export_to_video, load_image

pipe = I2VGenXLPipeline.from_pretrained(
    "ali-vilab/i2vgen-xl", torch_dtype=torch.float16, variant="fp16"
)

# Optimize
pipe.enable_model_cpu_offload()
pipe.unet.enable_forward_chunking()

image_url = (
    "/content/LSVD/reference.png"
    #"https://huggingface.co/datasets/diffusers/docs-images/resolve/main/i2vgen_xl_images/img_0009.png" #TODO change
)
image = load_image(image_url).convert("RGB")

generator = torch.manual_seed(8888)

try:
  frames = pipe(
      prompt=prompt,
      image=image,
      num_frames=num_frames,
      decode_chunk_size=decode_chunk_size,
      num_inference_steps=50,
      negative_prompt=negt,
      guidance_scale=9.0,
      generator=generator,
  ).frames[0]
  export_to_video(frames, "i2v.mp4", fps=fps)
except Exception as e:
  # cleanup

  pipe.to("cpu")
  del pipe
  torch.cuda.empty_cache()
  raise Exception(e)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

image_encoder/config.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

...

unet/config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Exception: CUDA out of memory. Tried to allocate 1.32 GiB. GPU 0 has a total capacity of 14.75 GiB of which 237.06 MiB is free. Process 139994 has 14.51 GiB memory in use. Of the allocated memory 14.25 GiB is allocated by PyTorch, and 136.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

------------- Text/Img to Video CogVideoX -------------

In [ ]:
import torch
from diffusers import CogVideoXImageToVideoPipeline
from diffusers.utils import export_to_video, load_image

image = load_image(image="/content/LSVD/reference.png") #TODO change
pipe = CogVideoXImageToVideoPipeline.from_pretrained(
    "THUDM/CogVideoX-5b-I2V",
    torch_dtype=torch.bfloat16
)

pipe.vae.enable_tiling()
pipe.vae.enable_slicing()

# Optimize
pipe.enable_model_cpu_offload()
#pipe.unet.enable_forward_chunking()

try:
  video = pipe(
      prompt=prompt,
      image=image,
      #decode_chunk_size=decode_chunk_size,
      num_videos_per_prompt=1,
      num_inference_steps=50,
      num_frames=num_frames,
      guidance_scale=6,
      generator=torch.Generator(device="cuda").manual_seed(42),
  ).frames[0]

  export_to_video(video, "cogVidX.mp4", fps=fps)
except Exception as e:
  # cleanup

  pipe.to("cpu")
  del pipe
  torch.cuda.empty_cache()
  raise Exception(e)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

------------- Img to Video Stable Video Diffusion -------------

In [2]:
import torch
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16, variant="fp16"
)

# Optimize
pipe.enable_model_cpu_offload()
pipe.unet.enable_forward_chunking()

image = load_image(
    "/content/LSVD/reference.png"
    #"https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/svd/rocket.png"
    ) #TODO change
image = image.resize((1024, 576))

generator = torch.manual_seed(42)
try:
  frames = pipe(image, num_frames=num_frames, decode_chunk_size=8, generator=generator).frames[0]
  export_to_video(frames, "svd.mp4", fps=fps)
except Exception as e:
  # cleanup

  pipe.to("cpu")
  del pipe
  torch.cuda.empty_cache()
  raise Exception(e)

# Worked in colab

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

unet/config.json:   0%|          | 0.00/984 [00:00<?, ?B/s]

image_encoder/config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/196M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/3.05G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

------------- Text to Video AnimateDiff -------------

In [2]:
import torch
from diffusers import AnimateDiffPipeline, DDIMScheduler, MotionAdapter
from diffusers.utils import export_to_video

adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16)

pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=torch.float16)
scheduler = DDIMScheduler.from_pretrained(
    "emilianJR/epiCRealism",
    subfolder="scheduler",
    clip_sample=False,
    timestep_spacing="linspace",
    beta_schedule="linear",
    steps_offset=1,
)
pipe.scheduler = scheduler
pipe.enable_vae_slicing()

# Optimize
pipe.enable_model_cpu_offload()
pipe.unet.enable_forward_chunking()

try:
  output = pipe(
      prompt=prompt,
      negative_prompt=negt,
      num_frames=32,
      decode_chunk_size=decode_chunk_size,
      guidance_scale=7.5,
      num_inference_steps=50,
      generator=torch.Generator("cpu").manual_seed(49),
  )
  frames = output.frames[0]
  export_to_video(frames, "animateDiff.mp4", fps=fps)
except Exception as e:
  # cleanup

  pipe.to("cpu")
  del pipe
  torch.cuda.empty_cache()
  raise Exception(e)

# Worked in colab

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

------------- Text to Video ModelscopeT2V -------------

In [4]:
import torch
from diffusers import DiffusionPipeline
from diffusers.utils import export_to_video

pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.enable_vae_slicing()

# Optimize
pipe.enable_model_cpu_offload()
pipe.unet.enable_forward_chunking()

try:
  video_frames = pipe(
      prompt,
      num_frames=num_frames,
      ).frames[0]
  export_to_video(video_frames, "modelscopet2v.mp4", fps=fps)
except Exception as e:
  # cleanup

  pipe.to("cpu")
  del pipe
  torch.cuda.empty_cache()
  raise Exception(e)

# Worked on Colab -> quality is not so good

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]